In [1]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.1/302.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00


In [2]:
# Log in to your W&B account
import wandb

# Use wandb-core
wandb.require("core")

In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import random

# Launch 5 simulated experiments
total_runs = 5
for run in range(total_runs):
  # 🐝 1️⃣ Start a new run to track this script
  wandb.init(
      # Set the project where this run will be logged
      project="basic-intro",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{run}",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.02,
      "architecture": "CNN",
      "dataset": "CIFAR-100",
      "epochs": 10,
      })

  # This simple block simulates a training loop logging metrics
  epochs = 10
  offset = random.random() / 5
  for epoch in range(2, epochs):
      acc = 1 - 2 ** -epoch - random.random() / epoch - offset
      loss = 2 ** -epoch + random.random() / epoch + offset

      # 🐝 2️⃣ Log metrics from your script to W&B
      wandb.log({"acc": acc, "loss": loss})

  # Mark the run as finished
  wandb.finish()

wandb: Currently logged in as: cmihadi (cmihadi-usiu). Use `wandb login --relogin` to force relogin


acc,▃▁▄▇▇██▇
loss,█▇▃▃▄▂▃▁
acc,0.79489
loss,0.1336


acc,▁▄▅▅▆███
loss,█▆▄▃▁▁▁▂
acc,0.78435
loss,0.25336


acc,▂▁▅▆▇▇██
loss,█▆▂▅▄▃▂▁
acc,0.82503
loss,0.17151


acc,▁▄▇▅█▇█▇
loss,█▅▂▃▂▂▁▂
acc,0.77729
loss,0.24887


acc,▁▇█▆██▇█
loss,█▅▂▂▃▁▁▁
acc,0.86999
loss,0.0916


3️⃣ You can find your interactive dashboard by clicking any of the  👆 wandb links above.

In [5]:
#@title
import wandb
import math
import random
import torch, torchvision
import torch.nn as nn
import torchvision.transforms as T

device = "cuda:0" if torch.cuda.is_available() else "cpu"

def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    full_dataset = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    sub_dataset = torch.utils.data.Subset(full_dataset, indices=range(0, len(full_dataset), slice))
    loader = torch.utils.data.DataLoader(dataset=sub_dataset,
                                         batch_size=batch_size,
                                         shuffle=True if is_train else False,
                                         pin_memory=True, num_workers=2)
    return loader

def get_model(dropout):
    "A simple model"
    model = nn.Sequential(nn.Flatten(),
                         nn.Linear(28*28, 256),
                         nn.BatchNorm1d(256),
                         nn.ReLU(),
                         nn.Dropout(dropout),
                         nn.Linear(256,10)).to(device)
    return model

def validate_model(model, valid_dl, loss_func, log_images=False, batch_idx=0):
    "Compute performance of the model on the validation dataset and log a wandb.Table"
    model.eval()
    val_loss = 0.
    with torch.inference_mode():
        correct = 0
        for i, (images, labels) in enumerate(valid_dl):
            images, labels = images.to(device), labels.to(device)

            # Forward pass ➡
            outputs = model(images)
            val_loss += loss_func(outputs, labels)*labels.size(0)

            # Compute accuracy and accumulate
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum().item()

            # Log one batch of images to the dashboard, always same batch_idx.
            if i==batch_idx and log_images:
                log_image_table(images, predicted, labels, outputs.softmax(dim=1))
    return val_loss / len(valid_dl.dataset), correct / len(valid_dl.dataset)

def log_image_table(images, predicted, labels, probs):
    "Log a wandb.Table with (img, pred, target, scores)"
    # 🐝 Create a wandb Table to log images, labels and predictions to
    table = wandb.Table(columns=["image", "pred", "target"]+[f"score_{i}" for i in range(10)])
    for img, pred, targ, prob in zip(images.to("cpu"), predicted.to("cpu"), labels.to("cpu"), probs.to("cpu")):
        table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
    wandb.log({"predictions_table":table}, commit=False)

In [6]:
# Launch 5 experiments, trying different dropout rates
for _ in range(5):
    # 🐝 initialise a wandb run
    wandb.init(
        project="pytorch-intro",
        config={
            "epochs": 10,
            "batch_size": 128,
            "lr": 1e-3,
            "dropout": random.uniform(0.01, 0.80),
            })

    # Copy your config
    config = wandb.config

    # Get the data
    train_dl = get_dataloader(is_train=True, batch_size=config.batch_size)
    valid_dl = get_dataloader(is_train=False, batch_size=2*config.batch_size)
    n_steps_per_epoch = math.ceil(len(train_dl.dataset) / config.batch_size)

    # A simple MLP model
    model = get_model(config.dropout)

    # Make the loss and optimizer
    loss_func = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

   # Training
    example_ct = 0
    step_ct = 0
    for epoch in range(config.epochs):
        model.train()
        for step, (images, labels) in enumerate(train_dl):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            train_loss = loss_func(outputs, labels)
            optimizer.zero_grad()
            train_loss.backward()
            optimizer.step()

            example_ct += len(images)
            metrics = {"train/train_loss": train_loss,
                       "train/epoch": (step + 1 + (n_steps_per_epoch * epoch)) / n_steps_per_epoch,
                       "train/example_ct": example_ct}

            if step + 1 < n_steps_per_epoch:
                # 🐝 Log train metrics to wandb
                wandb.log(metrics)

            step_ct += 1

        val_loss, accuracy = validate_model(model, valid_dl, loss_func, log_images=(epoch==(config.epochs-1)))

        # 🐝 Log train and validation metrics to wandb
        val_metrics = {"val/val_loss": val_loss,
                       "val/val_accuracy": accuracy}
        wandb.log({**metrics, **val_metrics})

        print(f"Train Loss: {train_loss:.3f}, Valid Loss: {val_loss:3f}, Accuracy: {accuracy:.2f}")

    # If you had a test set, this is how you could log it as a Summary metric
    wandb.summary['test_accuracy'] = 0.8

    # 🐝 Close your wandb run
    wandb.finish()

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15902703.98it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 482668.09it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



...

predictions_table/ncols,▁
predictions_table/nrows,▁
predictions_table/size,▁
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇█
train/example_ct,▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▆▆▆▇▇▇▇▇▇████
train/train_loss,█▆▄▃▄▂▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▂▁▁▁▁▁▁▁▁▂▁▂
val/val_accuracy,▁▃▄▆▆▆▇██▇
val/val_loss,█▅▄▃▂▁▁▁▁▁
test_accuracy,0.8
train/epoch,10
train/example_ct,120000


Train Loss: 0.486, Valid Loss: 0.352488, Accuracy: 0.90
Train Loss: 0.314, Valid Loss: 0.289077, Accuracy: 0.92
Train Loss: 0.377, Valid Loss: 0.258852, Accuracy: 0.93
Train Loss: 0.143, Valid Loss: 0.244451, Accuracy: 0.93
Train Loss: 0.293, Valid Loss: 0.229330, Accuracy: 0.93
Train Loss: 0.357, Valid Loss: 0.221462, Accuracy: 0.93
Train Loss: 0.265, Valid Loss: 0.214768, Accuracy: 0.93
Train Loss: 0.173, Valid Loss: 0.202814, Accuracy: 0.93
Train Loss: 0.253, Valid Loss: 0.204240, Accuracy: 0.94
Train Loss: 0.145, Valid Loss: 0.199683, Accuracy: 0.94


predictions_table/ncols,▁
predictions_table/nrows,▁
predictions_table/size,▁
train/epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
train/example_ct,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇███
train/train_loss,█▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▂▂▂▂▁▁▁▁▁▁▁▂▁▁▂▂▂▂
val/val_accuracy,▁▄▅▆▇▆▆▇█▇
val/val_loss,█▅▄▃▂▂▂▁▁▁
test_accuracy,0.8
train/epoch,10
train/example_ct,120000


Train Loss: 0.373, Valid Loss: 0.339727, Accuracy: 0.91
Train Loss: 0.314, Valid Loss: 0.273320, Accuracy: 0.92
Train Loss: 0.540, Valid Loss: 0.246995, Accuracy: 0.92
Train Loss: 0.319, Valid Loss: 0.223108, Accuracy: 0.94
Train Loss: 0.182, Valid Loss: 0.214282, Accuracy: 0.94
Train Loss: 0.302, Valid Loss: 0.204410, Accuracy: 0.94
Train Loss: 0.194, Valid Loss: 0.197084, Accuracy: 0.94
Train Loss: 0.269, Valid Loss: 0.187777, Accuracy: 0.94
Train Loss: 0.163, Valid Loss: 0.181930, Accuracy: 0.94
Train Loss: 0.135, Valid Loss: 0.180724, Accuracy: 0.95


predictions_table/ncols,▁
predictions_table/nrows,▁
predictions_table/size,▁
train/epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▆▆▇▇▇▇▇▇▇▇▇████
train/example_ct,▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇█████
train/train_loss,█▂▃▃▂▂▂▂▂▂▂▂▂▁▂▁▂▂▁▂▁▁▁▁▁▁▂▁▁▁▂▁▁▂▁▁▁▁▁▂
val/val_accuracy,▁▄▄▆▆▆▇▇▇█
val/val_loss,█▅▄▃▂▂▂▁▁▁
test_accuracy,0.8
train/epoch,10
train/example_ct,120000


You have now trained your first model using wandb! 👆 Click on the wandb link above to see your metrics

In [7]:
# Start a wandb run
wandb.init(project="pytorch-intro")

# Simulating a model training loop
acc_threshold = 0.3
for training_step in range(1000):

    # Generate a random number for accuracy
    accuracy = round(random.random() + random.random(), 3)
    print(f'Accuracy is: {accuracy}, {acc_threshold}')

    # 🐝 Log accuracy to wandb
    wandb.log({"Accuracy": accuracy})

    # 🔔 If the accuracy is below the threshold, fire a W&B Alert and stop the run
    if accuracy <= acc_threshold:
        # 🐝 Send the wandb Alert
        wandb.alert(
            title='Low Accuracy',
            text=f'Accuracy {accuracy} at step {training_step} is below the acceptable theshold, {acc_threshold}',
        )
        print('Alert triggered')
        break

# Mark the run as finished (useful in Jupyter notebooks)
wandb.finish()

Accuracy is: 0.971, 0.3
Accuracy is: 1.011, 0.3
Accuracy is: 1.338, 0.3
Accuracy is: 1.498, 0.3
Accuracy is: 1.071, 0.3
Accuracy is: 1.133, 0.3
Accuracy is: 1.257, 0.3
Accuracy is: 0.58, 0.3
Accuracy is: 0.57, 0.3
Accuracy is: 0.949, 0.3
Accuracy is: 0.556, 0.3
Accuracy is: 1.144, 0.3
Accuracy is: 1.222, 0.3
Accuracy is: 1.128, 0.3
Accuracy is: 0.308, 0.3
Accuracy is: 1.585, 0.3
Accuracy is: 0.886, 0.3
Accuracy is: 1.044, 0.3
Accuracy is: 1.052, 0.3
Accuracy is: 1.421, 0.3
Accuracy is: 0.899, 0.3
Accuracy is: 1.311, 0.3
Accuracy is: 1.186, 0.3
Accuracy is: 1.461, 0.3
Accuracy is: 0.88, 0.3
Accuracy is: 1.284, 0.3
Accuracy is: 0.402, 0.3
Accuracy is: 0.511, 0.3
Accuracy is: 1.5, 0.3
Accuracy is: 1.749, 0.3
Accuracy is: 1.392, 0.3
Accuracy is: 0.693, 0.3
Accuracy is: 1.232, 0.3
Accuracy is: 1.024, 0.3
Accuracy is: 0.466, 0.3
Accuracy is: 1.751, 0.3
Accuracy is: 1.396, 0.3
Accuracy is: 1.341, 0.3
Accuracy is: 1.3, 0.3
Accuracy is: 1.098, 0.3
Accuracy is: 1.541, 0.3
Accuracy is: 1.532, 0.3

Accuracy,▄▅▆▄▄▂▂▄▂▄▄▇▃▄▆▅▆▃▅▁▄▁▇▆▅▆▆▆█▂▆▆▄▅▂▃▆▅▄▂
Accuracy,0.166
